In [1]:
#ALL libraries

In [7]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [9]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import os
import warnings
from torch.cuda.amp import autocast, GradScaler

In [3]:
#Warning handles

In [11]:
warnings.filterwarnings("ignore", message="Passing a tuple of `past_key_values` is deprecated")

In [5]:
#data preprocess

In [13]:
qa_pairs = [
    ("Answer this question: How long does laundry take?", 
     "It typically takes 4 to 5 days to process and complete a laundry order. Check the app for your order’s estimated completion date."),

    ("Answer this question: What is the time taken for laundry?", 
     "Laundry usually takes 4 to 5 days. You can track your order progress in the app's Track Order section."),

    ("Answer this question: laundry kitne dino mein milega?", 
     "Aapka laundry order usually 4 se 5 din mein complete ho jata hai. Track Order page se aap status check kar sakte ho.")
]


In [ ]:
#with open("Datasets//laundry_faq_500_basic.json", "r") as file:
#    dataset = json.load(file)

# Convert to input-output format for training
#qa_pairs = [("Answer this question: " + item["question"], item["answer"]) for item in dataset]


# Load pre-trained tokenizer (FLAN-T5 tokenizer)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

# Tokenize data
def preprocess_data(pairs, tokenizer, max_length=128):
    inputs = []
    outputs = []
    
    for question, answer in pairs:
        input_enc = tokenizer(question, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
        output_enc = tokenizer(answer, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
        
        input_ids = input_enc["input_ids"].squeeze()
        output_ids = output_enc["input_ids"].squeeze()
        output_ids[output_ids == tokenizer.pad_token_id] = -100  # ignore loss on padding tokens

        inputs.append(input_ids)
        outputs.append(output_ids)

    return torch.stack(inputs), torch.stack(outputs)

# Prepare tokenized dataset
inputs, outputs = preprocess_data(qa_pairs, tokenizer)

In [8]:
#model intializing

In [17]:
class LaundryBot(nn.Module):
    def __init__(self):
        super(LaundryBot, self).__init__()
        self.model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

    def forward(self, input_ids, labels=None):
        return self.model(input_ids=input_ids, labels=labels)

# Initialize model
model = LaundryBot()

In [10]:
#training the model on dataset

In [19]:
# Define parameters
batch_size = 3
learning_rate = 5e-5
num_epochs = 30
device = torch.device("cuda")

# Create DataLoader
dataset = TensorDataset(inputs, outputs)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True,num_workers=4, pin_memory=True)

# Optimizer
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Load model checkpoint if exists
#model_filename = f"flan_laundry_bot1_{batch_size}_lr{learning_rate}.pth"
model_filename = "flan_laundry_bot1_8_lr5e-05.pth"
if os.path.exists(model_filename):
    print(f"Loading model from checkpoint: {model_filename}")
    model.load_state_dict(torch.load(model_filename))

# Train the model
model.to(device)

scaler = GradScaler()  # Place this before your main training loop

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    epoch_progress = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch", ncols=100)
    
    for batch in epoch_progress:
        input_ids, labels = batch
        input_ids, labels = input_ids.to(device), labels.to(device)
        
        optimizer.zero_grad()
        with autocast():
            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
        epoch_progress.set_postfix(loss=total_loss / (epoch_progress.n + 1))
        
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

# Save model after training
torch.save(model.state_dict(), model_filename)
print(f"Model saved as {model_filename}")


Loading model from checkpoint: flan_laundry_bot1_8_lr5e-05.pth


C:\Users\keshu\AppData\Local\Temp\ipykernel_21152\44815235.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_filename))
C:\Users\ke

Epoch 1, Loss: nan


Epoch 2/30: 100%|████████████████████████████████████████| 1/1 [00:02<00:00,  2.44s/batch, loss=nan]


Epoch 2, Loss: nan


...

Epoch 28, Loss: nan


Epoch 29/30: 100%|███████████████████████████████████████| 1/1 [00:02<00:00,  2.51s/batch, loss=nan]


Epoch 29, Loss: nan


Epoch 30/30: 100%|███████████████████████████████████████| 1/1 [00:02<00:00,  2.50s/batch, loss=nan]


Epoch 30, Loss: nan
Model saved as flan_laundry_bot1_8_lr5e-05.pth


In [12]:
#testing of bot

In [21]:
# Load trained model for inference
model.load_state_dict(torch.load("flan_laundry_bot1_8_lr5e-05.pth"))
model.eval()

# Chatbot response function
def get_response(query):
    query = "Answer this question: " + query
    input_ids = tokenizer(query, return_tensors="pt").input_ids.to(device)
    output_ids = model.model.generate(input_ids, max_length=50)
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

# Example interaction
user_query = "How long will my laundry take?"
bot_reply = get_response(user_query)
print(f"Bot: {bot_reply}")


C:\Users\keshu\AppData\Local\Temp\ipykernel_21152\959033687.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("flan_laundry_bot1_8_lr5e-05

Bot: Your order will typically take 4 to 5 days to complete.


In [35]:
test_questions = [
    # ✅ Laundry Order Placement
    "How do I place a laundry order?",
    "Where can I place my clothes for laundry?",
    "Can I place an order anytime?",
    "Is there a limit to how many clothes I can order?",
    "What types of clothes can I include in my laundry order?",
    "How to place an urgent laundry order?",

    # 🧼 Dry Clean Specific
    "How do I place a dry clean order?",
    "What is the price of dry cleaning a blanket?",
    "Which items are allowed in dry clean?",
    "Can I add dry clean and normal wash in one order?",
    "Is dry cleaning available on weekends?",

    # 📦 Tracking Orders
    "How can I track my laundry order?",
    "What does 'ongoing' status mean?",
    "How long will my laundry take?",
    "Where can I see order status?",
    "Can I cancel an ongoing order?",

    # 🧾 Order History
    "Where can I see my past orders?",
    "How to view laundry history?",
    "Can I check previous dry clean orders?",
    "Do I have access to all completed orders?",
    "Can I see which items I ordered last time?",

    # 💰 Payments
    "How to pay for my laundry?",
    "What are the payment options?",
    "Can I use Paytm for laundry payment?",
    "Where can I see pending payment?",
    "Why is my payment marked pending?",
    "How to check my payment history?",

    # 👤 App & User Info
    "Who made this laundry app?",
    "Is this app available for staff too?",
    "Can I use this app on iPhone?",
    "Is the app only for Bennett students?",
    "What happens if I forget my password?",

    # ⚠️ Issues & Support
    "My order is stuck in processing, what should I do?",
    "I paid but the status is still pending.",
    "I selected the wrong clothes, can I change it?",
    "My order was not picked up, whom do I contact?",
    "Can I get a receipt for my payment?"
]


for idx, question in enumerate(test_questions, 1):
    bot_reply = get_response(question)
    print(f"{idx}. Q: {question}\n   Bot: {bot_reply}\n")

1. Q: How do I place a laundry order?
   Bot: Go to the 'Place Order' page, select items you want cleaned, and tap the button at the bottom to confirm your order.

2. Q: Where can I place my clothes for laundry?
   Bot: You can place your clothes in the laundry room, or you can place them in the drawers at the bottom of the screen.

3. Q: Can I place an order anytime?
   Bot: Yes, you can place an order anytime from the 'Place Order' screen.

4. Q: Is there a limit to how many clothes I can order?
   Bot: No, I can order up to 10 clothes per day.

5. Q: What types of clothes can I include in my laundry order?
   Bot: You can include any type of clothes, such as jeans, for your laundry order.

6. Q: How to place an urgent laundry order?
   Bot: Go to the 'Place Order' page, select the items you want cleaned, and press the 'Place Order' button.

7. Q: How do I place a dry clean order?
   Bot: Go to the 'Dry Clean' page, select items for dry cleaning, and press the 'Dry Clean' button.

8.

In [14]:
#if using gpu and memory(load)
############################del model
#torch.cuda.empty_cache()  # If using GPU
